In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

To get Tennessee and the US you have to download from the , each of which you will save with the listed file names as .csv's after deleting the top 3 rows with the industry information and the bottom rows containing the data notes: *THIS IS FROM THE 2017 BASE FILES*

+ WP032: Total Employment --> (WP2017_TotalEmployment.csv)
+ WP033: Farm Employment --> (WP2017_FarmEmployment.csv)  
+ WP034: Forestry, Fishing, Related Activities & Other Employment --> (WP2017_ForestryFishingandRelatedEmployment.csv)
+ WP035: Mining Employment --> (WP2017_MiningEmployment.csv)  
+ WP036: Utilities Employment --> (WP2017_UtilitiesEmployment.csv)  
+ WP037: Construction Employment --> (WP2017_ConstructionEmployment.csv')  
+ WP038: Manufacturing Employment --> (WP2017_ManufacturingEmployment.csv')  
+ WP039: Wholesale Trade Employment --> (WP2017_WholesaleTradeEmployment.csv)  
+ WP040: Retail Trade Employment --> (WP2017_RetailTradeEmployment.csv)  
+ WP041: Transportation & Warehousing Employment --> (WP2017_TransportationandWarehousingEmployment.csv)  
+ WP042: Information Employment --> (WP2017_InformationEmployment.csv)  
+ WP043: Finance & Insurance Employment --> (WP2017_FinanceandInsuranceEmployment.csv)  
+ WP044: Real Estate & Rental & Leasing Employment --> (WP2017_RealEstateandRentalandLeasingEmployment.csv)  
+ WP045: Professional & Technical Services Employment --> (WP2017_ProfessionalandTechnicalServicesEmployment.csv)  
+ WP046: Management of Companies & Enterprises Employment --> (WP2017_ManagementofCompaniesandEnterprisesEmployment.csv)  
+ WP047: Administrative & Waste Services Employment --> (WP2017_AdministrativeandWasteServicesEmployment.csv)  
+ WP048: Educational Services Employment --> (WP2017_EducationalServicesEmployment.csv)  
+ WP049: Healthcare & Social Assistance Employment --> (WP2017_HealthcareandSocialAssistanceEmployment.csv)  
+ WP050: Arts, Entertainment, & Recreation Employment --> (WP2017_ArtsEntertainmentandRecreationEmployment.csv)  
+ WP051: Accommodation & Food Services Employment --> (WP2017_AccommodationandFoodServicesEmployment.csv)  
+ WP052: Other Services, Except Public Administration Employment --> (WP2017_OtherExceptPublicAdministrationEmployment.csv)  
+ WP053: Federal Civilian Government Employment --> (WP2017_FederalCivilianGovernmentEmployment.csv)  
+ WP054: Federal Military Employment --> (WP2017_FederalMilitaryEmployment.csv)  
+ WP055: State & Local Government Employment --> (WP2017_StateandLocalGovernmentEmployment.csv)  

Also bring in the UrbanSIM data for total employment at the place level so that this can all just be one .csv

In [2]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

## NAICS-Level files for US, Tennessee, and Counties

In [3]:
totalemployment = pd.read_csv('../../Data Downloads/WP2017_TotalEmployment.csv')
farm = pd.read_csv('../../Data Downloads/WP2017_FarmEmployment.csv') 
forestry_fishing_related = pd.read_csv('../../Data Downloads/WP2017_ForestryFishingandRelatedEmployment.csv')
mining = pd.read_csv('../../Data Downloads/WP2017_MiningEmployment.csv')  
utilities = pd.read_csv('../../Data Downloads/WP2017_UtilitiesEmployment.csv')  
construction = pd.read_csv('../../Data Downloads/WP2017_ConstructionEmployment.csv') 
manufacturing = pd.read_csv('../../Data Downloads/WP2017_ManufacturingEmployment.csv') 
wholesale_trade = pd.read_csv('../../Data Downloads/WP2017_WholesaleTradeEmployment.csv') 
retail_trade = pd.read_csv('../../Data Downloads/WP2017_RetailTradeEmployment.csv') 
transportation_warehousing = pd.read_csv('../../Data Downloads/WP2017_TransportationandWarehousingEmployment.csv')  
information = pd.read_csv('../../Data Downloads/WP2017_InformationEmployment.csv') 
finance_insurance = pd.read_csv('../../Data Downloads/WP2017_FinanceandInsuranceEmployment.csv')
realestate_rental_leasing = pd.read_csv('../../Data Downloads/WP2017_RealEstateandRentalandLeasingEmployment.csv') 
professional_technical_services = pd.read_csv('../../Data Downloads/WP2017_ProfessionalandTechnicalServicesEmployment.csv') 
management_companies_enterprises = pd.read_csv('../../Data Downloads/WP2017_ManagementofCompaniesandEnterprisesEmployment.csv')  
administrative_waste_services = pd.read_csv('../../Data Downloads/WP2017_AdministrativeandWasteServicesEmployment.csv') 
educational_services = pd.read_csv('../../Data Downloads/WP2017_EducationalServicesEmployment.csv')  
healthcare_social_assistance = pd.read_csv('../../Data Downloads/WP2017_HealthcareandSocialAssistanceEmployment.csv')
arts_entertainment_recreation = pd.read_csv('../../Data Downloads/WP2017_ArtsEntertainmentandRecreationEmployment.csv') 
accommodation_food_services = pd.read_csv('../../Data Downloads/WP2017_AccommodationandFoodServicesEmployment.csv')  
other_except_public_administration = pd.read_csv('../../Data Downloads/WP2017_OtherExceptPublicAdministrationEmployment.csv')
federal_civilian = pd.read_csv('../../Data Downloads/WP2017_FederalCivilianGovernmentEmployment.csv')  
federal_military = pd.read_csv('../../Data Downloads/WP2017_FederalMilitaryEmployment.csv')  
state_local = pd.read_csv('../../Data Downloads/WP2017_StateandLocalGovernmentEmployment.csv') 

In [213]:
dfs = [totalemployment, farm, forestry_fishing_related, mining, utilities, construction, manufacturing, wholesale_trade, retail_trade, 
       transportation_warehousing, information, finance_insurance, realestate_rental_leasing, professional_technical_services, 
       management_companies_enterprises, administrative_waste_services, educational_services, healthcare_social_assistance, arts_entertainment_recreation, 
       accommodation_food_services, other_except_public_administration, federal_civilian, federal_military, state_local]

In [214]:
totalemployment = totalemployment.loc[(totalemployment['NAME'] == 'UNITED STATES')|(totalemployment['NAME'] == 'TENNESSEE')|(totalemployment['TYPE'] == '5')]
farm = farm.loc[(farm['NAME'] == 'UNITED STATES')|(farm['NAME'] == 'TENNESSEE')|(farm['TYPE'] == '5')]
forestry_fishing_related = forestry_fishing_related.loc[(forestry_fishing_related['NAME'] == 'UNITED STATES')|(forestry_fishing_related['NAME'] == 'TENNESSEE')|
                                                        (forestry_fishing_related['TYPE'] == '5')]
mining = mining.loc[(mining['NAME'] == 'UNITED STATES')|(mining['NAME'] == 'TENNESSEE')|(mining['TYPE'] == '5')]
utilities = utilities.loc[(utilities['NAME'] == 'UNITED STATES')|(utilities['NAME'] == 'TENNESSEE')|(utilities['TYPE'] == '5')]
construction = construction.loc[(construction['NAME'] == 'UNITED STATES')|(construction['NAME'] == 'TENNESSEE')|(construction['TYPE'] == '5')]
manufacturing = manufacturing.loc[(manufacturing['NAME'] == 'UNITED STATES')|(manufacturing['NAME'] == 'TENNESSEE')|(manufacturing['TYPE'] == '5')]
wholesale_trade = wholesale_trade.loc[(wholesale_trade['NAME'] == 'UNITED STATES')|(wholesale_trade['NAME'] == 'TENNESSEE')|(wholesale_trade['TYPE'] == '5')]
retail_trade = retail_trade.loc[(retail_trade['NAME'] == 'UNITED STATES')|(retail_trade['NAME'] == 'TENNESSEE')|(retail_trade['TYPE'] == '5')]
transportation_warehousing = transportation_warehousing.loc[(transportation_warehousing['NAME'] == 'UNITED STATES')|
                                                            (transportation_warehousing['NAME'] == 'TENNESSEE')|(transportation_warehousing['TYPE'] == '5')]
information = information.loc[(information['NAME'] == 'UNITED STATES')|(information['NAME'] == 'TENNESSEE')|(information['TYPE'] == '5')]
finance_insurance = finance_insurance.loc[(finance_insurance['NAME'] == 'UNITED STATES')|(finance_insurance['NAME'] == 'TENNESSEE')|(finance_insurance['TYPE'] == '5')]
realestate_rental_leasing = realestate_rental_leasing.loc[(realestate_rental_leasing['NAME'] == 'UNITED STATES')|(realestate_rental_leasing['NAME'] == 'TENNESSEE')|
                                                          (realestate_rental_leasing['TYPE'] == '5')]
professional_technical_services = professional_technical_services.loc[(professional_technical_services['NAME'] == 'UNITED STATES')|
                                                                      (professional_technical_services['NAME'] == 'TENNESSEE')|
                                                                      (professional_technical_services['TYPE'] == '5')]
management_companies_enterprises = management_companies_enterprises.loc[(management_companies_enterprises['NAME'] == 'UNITED STATES')|
                                                                        (management_companies_enterprises['NAME'] == 'TENNESSEE')|
                                                                        (management_companies_enterprises['TYPE'] == '5')]
administrative_waste_services = administrative_waste_services.loc[(administrative_waste_services['NAME'] == 'UNITED STATES')|
                                                                  (administrative_waste_services['NAME'] == 'TENNESSEE')|
                                                                  (administrative_waste_services['TYPE'] == '5')]
educational_services = educational_services.loc[(educational_services['NAME'] == 'UNITED STATES')|(educational_services['NAME'] == 'TENNESSEE')|
                                                (educational_services['TYPE'] == '5')]
healthcare_social_assistance = healthcare_social_assistance.loc[(healthcare_social_assistance['NAME'] == 'UNITED STATES')|
                                                                (healthcare_social_assistance['NAME'] == 'TENNESSEE')|(healthcare_social_assistance['TYPE'] == '5')]
arts_entertainment_recreation = arts_entertainment_recreation.loc[(arts_entertainment_recreation['NAME'] == 'UNITED STATES')|
                                                                  (arts_entertainment_recreation['NAME'] == 'TENNESSEE')|
                                                                  (arts_entertainment_recreation['TYPE'] == '5')]
accommodation_food_services = accommodation_food_services.loc[(accommodation_food_services['NAME'] == 'UNITED STATES')|
                                                              (accommodation_food_services['NAME'] == 'TENNESSEE')|(accommodation_food_services['TYPE'] == '5')]
other_except_public_administration = other_except_public_administration.loc[(other_except_public_administration['NAME'] == 'UNITED STATES')|
                                                                            (other_except_public_administration['NAME'] == 'TENNESSEE')|
                                                                            (other_except_public_administration['TYPE'] == '5')]
federal_civilian = federal_civilian.loc[(federal_civilian['NAME'] == 'UNITED STATES')|(federal_civilian['NAME'] == 'TENNESSEE')|(federal_civilian['TYPE'] == '5')]
federal_military = federal_military.loc[(federal_military['NAME'] == 'UNITED STATES')|(federal_military['NAME'] == 'TENNESSEE')|(federal_military['TYPE'] == '5')]
state_local = state_local.loc[(state_local['NAME'] == 'UNITED STATES')|(state_local['NAME'] == 'TENNESSEE')|(state_local['TYPE'] == '5')]

In [215]:
thelist = ['FILE', 'COUNTY', 'TYPE', 'REGION', 'EA', 'MSA', 'CSA', 'MDIV', 'MICRO', '1969', '1970', '1971', '1972', '1973',
                                                  '1974', '1975', '1976', '1977', '1978', '1979']

In [216]:
totalemployment = totalemployment.drop(columns = thelist)
farm = farm.drop(columns = thelist)
forestry_fishing_related = forestry_fishing_related.drop(columns = thelist)
mining = mining.drop(columns = thelist)
utilities =   utilities.drop(columns = thelist)
construction = construction.drop(columns = thelist)
manufacturing =  manufacturing.drop(columns = thelist)
wholesale_trade = wholesale_trade.drop(columns = thelist)
retail_trade = retail_trade.drop(columns = thelist)
transportation_warehousing =  transportation_warehousing.drop(columns = thelist)
information = information.drop(columns = thelist)
finance_insurance = finance_insurance.drop(columns = thelist)
realestate_rental_leasing = realestate_rental_leasing.drop(columns = thelist)
professional_technical_services = professional_technical_services.drop(columns = thelist)
management_companies_enterprises = management_companies_enterprises.drop(columns = thelist)
administrative_waste_services = administrative_waste_services.drop(columns = thelist)
educational_services =  educational_services.drop(columns = thelist)
healthcare_social_assistance = healthcare_social_assistance.drop(columns = thelist)
arts_entertainment_recreation = arts_entertainment_recreation.drop(columns = thelist)
accommodation_food_services =  accommodation_food_services.drop(columns = thelist)
other_except_public_administration = other_except_public_administration.drop(columns = thelist)
federal_civilian =  federal_civilian.drop(columns = thelist)
federal_military =  federal_military.drop(columns = thelist)
state_local = state_local.drop(columns = thelist)

In [217]:
state_local.head()

,NAME,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,UNITED STATES,13280.000,13156.999,13080.000,13095.000,13249.000,13503.000,13806.000,14094.000,14456.000,14822.999,15281.000,15494.999,15718.000,15995.000,16286.001,16521.000,16682.999,16938.000,17246.000,17591.001,17977.000,18351.001,18689.000,18757.000,18861.000,19023.000,19182.000,19428.000,19707.000,19713.000,19537.000,19290.000,19171.000,19197.000,19279.000,19376.000,19661.325,19944.249,20223.518,20498.248,20767.993,21032.399,21290.511,21541.863,21786.273,22023.540,22253.680,22476.722,22692.716,22901.472,23102.995,23297.189,23484.045,23663.554,23835.717,24000.576,24158.464,24310.059,24455.709,24595.964,24731.239,24861.992,24988.787,25111.971,25231.860,25348.452,25461.917,25572.516,25680.581,25786.349,25889.978
2,TENNESSEE,258.687,249.769,245.581,246.776,246.430,248.585,257.185,266.405,270.870,276.768,281.762,285.531,293.809,302.585,311.999,315.056,321.531,324.489,329.954,336.421,341.993,349.062,355.579,357.680,364.264,364.947,370.344,374.132,380.297,377.882,380.441,379.422,375.969,375.091,373.932,371.541,376.527,381.465,386.340,391.118,395.808,400.409,404.897,409.261,413.512,417.633,421.645,425.525,429.287,432.922,436.439,439.835,443.110,446.257,449.280,452.184,454.969,457.653,460.233,462.719,465.128,467.462,469.721,471.923,474.061,476.137,478.167,480.144,482.068,483.965,485.813
37,"ANDERSON, TN",2.862,2.720,2.842,2.778,2.877,2.836,2.901,2.937,2.898,2.975,3.094,3.139,3.346,3.330,3.481,3.509,3.629,3.678,3.772,3.763,3.910,3.956,3.956,3.989,4.086,4.099,4.218,4.313,4.453,4.381,4.355,4.279,4.243,4.240,4.184,4.114,4.152,4.188,4.224,4.258,4.292,4.324,4.355,4.384,4.412,4.439,4.465,4.489,4.512,4.534,4.554,4.573,4.591,4.608,4.624,4.639,4.652,4.665,4.676,4.687,4.697,4.706,4.714,4.722,4.729,4.735,4.741,4.746,4.751,4.755,4.759
38,"BEDFORD, TN",1.456,1.398,1.401,1.377,1.431,1.411,1.398,1.428,1.490,1.541,1.638,1.681,1.670,1.778,1.778,1.816,1.891,2.010,1.929,2.109,2.169,2.237,2.304,2.285,2.328,2.422,2.107,2.113,2.013,2.200,2.282,2.276,2.179,2.362,2.261,2.344,2.370,2.396,2.421,2.446,2.469,2.492,2.515,2.536,2.557,2.576,2.595,2.613,2.630,2.646,2.661,2.675,2.689,2.701,2.713,2.724,2.734,2.743,2.752,2.760,2.767,2.773,2.779,2.785,2.790,2.794,2.798,2.801,2.804,2.807,2.809
39,"BENTON, TN",0.828,0.799,0.793,0.750,0.756,0.621,0.563,0.572,0.566,0.572,0.576,0.597,0.626,0.640,0.650,0.687,0.702,0.714,0.764,0.779,0.766,0.818,0.844,0.846,0.895,0.953,1.007,1.018,1.078,1.045,1.077,1.057,1.034,1.001,1.008,0.998,1.009,1.020,1.031,1.041,1.051,1.061,1.071,1.080,1.089,1.098,1.106,1.114,1.122,1.129,1.136,1.143,1.150,1.156,1.162,1.168,1.173,1.178,1.183,1.188,1.192,1.197,1.201,1.205,1.208,1.212,1.216,1.219,1.222,1.225,1.228


In [218]:
totalemployment = totalemployment.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Total')
farm = farm.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Farm')
forestry_fishing_related = forestry_fishing_related.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Forestry, Fishing, & Related')
mining = mining.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Mining')
utilities =   utilities.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Utilities')
construction = construction.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Construction')
manufacturing =  manufacturing.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Manufacturing')
wholesale_trade = wholesale_trade.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Wholesale Trade')
retail_trade = retail_trade.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Retail Trade')
transportation_warehousing =  transportation_warehousing.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Transportation & Warehousing')
information = information.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Information')
finance_insurance = finance_insurance.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Finance & Insurance')
realestate_rental_leasing = realestate_rental_leasing.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Real Estate, Rental, & Leasing')
professional_technical_services = professional_technical_services.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Professional & Technical Services')
management_companies_enterprises = management_companies_enterprises.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Management of Companies & Enterprises')
administrative_waste_services = administrative_waste_services.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Administrative & Waste Services')
educational_services =  educational_services.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Educational Services')
healthcare_social_assistance = healthcare_social_assistance.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Healthcare & Social Assistance')
arts_entertainment_recreation = arts_entertainment_recreation.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Arts, Entertainment, & Recreation')
accommodation_food_services =  accommodation_food_services.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Accommodation & Food Services')
other_except_public_administration = other_except_public_administration.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Other')
federal_civilian =  federal_civilian.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Federal Civilian')
federal_military =  federal_military.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'Federal Military')
state_local = state_local.melt(id_vars = 'NAME', var_name = 'Year', value_name = 'State & Local Government')

In [219]:
state_local.head()

,NAME,Year,State & Local Government
0,UNITED STATES,1980,13280.000
1,TENNESSEE,1980,258.687
2,"ANDERSON, TN",1980,2.862
3,"BEDFORD, TN",1980,1.456
4,"BENTON, TN",1980,0.828


In [220]:
dfs = [totalemployment, farm, forestry_fishing_related, mining, utilities, construction, manufacturing, wholesale_trade, retail_trade, 
       transportation_warehousing, information, finance_insurance, realestate_rental_leasing, professional_technical_services, 
       management_companies_enterprises, administrative_waste_services, educational_services, healthcare_social_assistance, arts_entertainment_recreation, 
       accommodation_food_services, other_except_public_administration, federal_civilian, federal_military, state_local]

In [221]:
df_merged = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'Year'], how='outer'), dfs)

In [222]:
FINALGEOS = ['UNITED STATES', 'TENNESSEE', 'CHEATHAM, TN', 'DAVIDSON, TN', 'DICKSON, TN', 'HOUSTON, TN', 'HUMPHREYS, TN', 'MAURY, TN', 'MONTGOMERY, TN', 
             'ROBERTSON, TN', 'RUTHERFORD, TN', 'STEWART, TN', 'SUMNER, TN', 'TROUSDALE, TN', 'WILLIAMSON, TN', 'WILSON, TN']

In [223]:
df_merged = df_merged.loc[df_merged['NAME'].isin(FINALGEOS)]

In [224]:
times = list(df_merged['Year'].unique())
#times = list(data['Year'].unique())

In [225]:
df_merged = df_merged.set_index(['NAME', 'Year']).transpose()

In [226]:
df_merged = df_merged.rename(columns = {'UNITED STATES': 'United States', 'TENNESSEE': 'Tennessee', 'CHEATHAM, TN': 'Cheatham County, Tennessee', 
                                        'DAVIDSON, TN': 'Davidson County, Tennessee', 'DICKSON, TN': 'Dickson County, Tennessee', 
                                        'HOUSTON, TN': 'Houston County, Tennessee', 'HUMPHREYS, TN': 'Humphreys County, Tennessee', 
                                        'MAURY, TN': 'Maury County, Tennessee', 'MONTGOMERY, TN': 'Montgomery County, Tennessee', 
                                        'ROBERTSON, TN': 'Robertson County, Tennessee', 'RUTHERFORD, TN': 'Rutherford County, Tennessee', 
                                        'STEWART, TN': 'Stewart County, Tennessee', 'SUMNER, TN': 'Sumner County, Tennessee', 
                                        'TROUSDALE, TN': 'Trousdale County, Tennessee', 'WILLIAMSON, TN': 'Williamson County, Tennessee', 
                                        'WILSON, TN': 'Wilson County, Tennessee'})

In [227]:
df_merged.head()

NAME,United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee",United States,Tennessee,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tenness

In [228]:
cols = df_merged.columns
df_merged[cols] = df_merged[cols].astype(float)

In [229]:
cols

MultiIndex([(               'United States', '1980'),
            (                   'Tennessee', '1980'),
            (  'Cheatham County, Tennessee', '1980'),
            (  'Davidson County, Tennessee', '1980'),
            (   'Dickson County, Tennessee', '1980'),
            (   'Houston County, Tennessee', '1980'),
            ( 'Humphreys County, Tennessee', '1980'),
            (     'Maury County, Tennessee', '1980'),
            ('Montgomery County, Tennessee', '1980'),
            ( 'Robertson County, Tennessee', '1980'),
            ...
            ( 'Humphreys County, Tennessee', '2050'),
            (     'Maury County, Tennessee', '2050'),
            ('Montgomery County, Tennessee', '2050'),
            ( 'Robertson County, Tennessee', '2050'),
            ('Rutherford County, Tennessee', '2050'),
            (   'Stewart County, Tennessee', '2050'),
            (    'Sumner County, Tennessee', '2050'),
            ( 'Trousdale County, Tennessee', '2050'),
            

In [230]:
data = df_merged

In [231]:
#aggregate by double column-headers
for time in times:
    #MPO
    thelist = [data[('Davidson County, Tennessee', '{}'.format(time))],data[('Maury County, Tennessee', '{}'.format(time))],
               data[('Robertson County, Tennessee', '{}'.format(time))],data[('Rutherford County, Tennessee', '{}'.format(time))], 
               data[('Sumner County, Tennessee', '{}'.format(time))], data[('Williamson County, Tennessee', '{}'.format(time))], 
               data[('Wilson County, Tennessee', '{}'.format(time))]]
    data[('MPO', '{}'.format(time))] = sum(thelist)
    #GNRC
    thelist = [data[('Cheatham County, Tennessee', '{}'.format(time))], data[('Davidson County, Tennessee', '{}'.format(time))], 
               data[('Dickson County, Tennessee', '{}'.format(time))], data[('Houston County, Tennessee', '{}'.format(time))],
               data[('Humphreys County, Tennessee', '{}'.format(time))], data[('Montgomery County, Tennessee', '{}'.format(time))],
               data[('Robertson County, Tennessee', '{}'.format(time))],data[('Rutherford County, Tennessee', '{}'.format(time))], 
               data[('Sumner County, Tennessee', '{}'.format(time))], data[('Stewart County, Tennessee', '{}'.format(time))], 
               data[('Trousdale County, Tennessee', '{}'.format(time))],
               data[('Williamson County, Tennessee', '{}'.format(time))], data[('Wilson County, Tennessee', '{}'.format(time))]]
    data[('GNRC', '{}'.format(time))] = sum(thelist)
    #GNRC Region
    thelist = [data[('Cheatham County, Tennessee', '{}'.format(time))], data[('Davidson County, Tennessee', '{}'.format(time))], 
               data[('Dickson County, Tennessee', '{}'.format(time))], data[('Houston County, Tennessee', '{}'.format(time))],
               data[('Humphreys County, Tennessee', '{}'.format(time))], data[('Montgomery County, Tennessee', '{}'.format(time))],
               data[('Robertson County, Tennessee', '{}'.format(time))],data[('Rutherford County, Tennessee', '{}'.format(time))], 
               data[('Sumner County, Tennessee', '{}'.format(time))], data[('Stewart County, Tennessee', '{}'.format(time))], 
               data[('Trousdale County, Tennessee', '{}'.format(time))], data[('Williamson County, Tennessee', '{}'.format(time))], 
               data[('Wilson County, Tennessee', '{}'.format(time))], data[('Maury County, Tennessee', '{}'.format(time))]]
    data[('GNRC Region', '{}'.format(time))] = sum(thelist)

In [232]:
df_merged = df_merged.transpose().reset_index(drop = False)

In [233]:
df_merged = df_merged.melt(id_vars = ['NAME', 'Year'], var_name = 'Industry', value_name = 'Employment')

In [234]:
df_merged.head()

,NAME,Year,Industry,Employment
0,United States,1980,Total,114118.464
1,Tennessee,1980,Total,2257.544
2,"Cheatham County, Tennessee",1980,Total,5.908
3,"Davidson County, Tennessee",1980,Total,319.165
4,"Dickson County, Tennessee",1980,Total,10.138


In [132]:
df_merged = df_merged.set_index(['Industry', 'Year', 'NAME']).transpose()

In [235]:
years = ['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', 
         '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', 
         '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', 
         '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2030', '2035', '2040', '2045']
geos = ['United States', 'Tennessee', 'Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 
        'Houston County, Tennessee', 'Humphreys County, Tennessee', 'Maury County, Tennessee', 'Montgomery County, Tennessee', 
        'Robertson County, Tennessee', 'Rutherford County, Tennessee', 'Stewart County, Tennessee', 'Sumner County, Tennessee', 
        'Trousdale County, Tennessee', 'Williamson County, Tennessee', 'Wilson County, Tennessee', 'MPO', 'GNRC', 'GNRC Region']

In [236]:
data = df_merged

In [237]:
cols = data.columns
for year in years:
    for geo in geos:
        data[('Total Employment', '{}'.format(year), '{}'.format(geo))] = data[('Total', '{}'.format(year), '{}'.format(geo))] * 1000
        data[('Farm Employment', '{}'.format(year), '{}'.format(geo))] = data[('Farm', '{}'.format(year), '{}'.format(geo))] * 1000
        thelist = [data[("Farm", '{}'.format(year), '{}'.format(geo))], 
                   data[('Forestry, Fishing, & Related', '{}'.format(year), '{}'.format(geo))], 
                   data[('Mining', '{}'.format(year), '{}'.format(geo))]]
        data[('Natural Resources & Mining Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

        data[('Construction Employment', '{}'.format(year), '{}'.format(geo))] = data[('Construction', '{}'.format(year), '{}'.format(geo))] * 1000

        data[('Manufacturing Employment', '{}'.format(year), '{}'.format(geo))] = data[('Manufacturing', '{}'.format(year), '{}'.format(geo))] * 1000
        thelist = [data[("Utilities", '{}'.format(year), '{}'.format(geo))], 
                   data[('Wholesale Trade', '{}'.format(year), '{}'.format(geo))], 
                   data[('Retail Trade', '{}'.format(year), '{}'.format(geo))], 
                   data[('Transportation & Warehousing', '{}'.format(year), '{}'.format(geo))]]
        data[('Trade, Transportation, & Utilities Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

        data[('Information Employment', '{}'.format(year), '{}'.format(geo))] = data[('Information', '{}'.format(year), '{}'.format(geo))] * 1000

        thelist = [data[("Finance & Insurance", '{}'.format(year), '{}'.format(geo))], 
                   data[('Real Estate, Rental, & Leasing', '{}'.format(year), '{}'.format(geo))]]
        data[('Financial Activities Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

        thelist = [data[("Professional & Technical Services", '{}'.format(year), '{}'.format(geo))], 
                   data[('Management of Companies & Enterprises', '{}'.format(year), '{}'.format(geo))], 
                   data[('Administrative & Waste Services', '{}'.format(year), '{}'.format(geo))]]
        data[('Professional & Business Services Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

        thelist = [data[("Educational Services", '{}'.format(year), '{}'.format(geo))], 
                   data[('Healthcare & Social Assistance', '{}'.format(year), '{}'.format(geo))]]
        data[('Education & Health Services Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

        thelist = [data[("Arts, Entertainment, & Recreation", '{}'.format(year), '{}'.format(geo))], 
                   data[('Accommodation & Food Services', '{}'.format(year), '{}'.format(geo))]]
        data[('Leisure & Hospitality Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

        data[('Other Employment', '{}'.format(year), '{}'.format(geo))] = data[('Other', '{}'.format(year), '{}'.format(geo))] * 1000

        thelist = [data[("Federal Civilian", '{}'.format(year), '{}'.format(geo))], 
                   data[('Federal Military', '{}'.format(year), '{}'.format(geo))], 
                   data[('State & Local Government', '{}'.format(year), '{}'.format(geo))]]
        data[('Public Administration Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000
        thelist = [data[('Natural Resources & Mining Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Construction Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Manufacturing Employment', '{}'.format(year), '{}'.format(geo))]]
        data[('Goods Producing Supersector Employment', '{}'.format(year), '{}'.format(geo))] = sum(thelist)
        thelist = [data[('Information Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Financial Activities Employment', '{}'.format(year), '{}'.format(geo))],
                   data[('Professional & Business Services Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Education & Health Services Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Leisure & Hospitality Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Other Employment', '{}'.format(year), '{}'.format(geo))], 
                   data[('Public Administration Employment', '{}'.format(year), '{}'.format(geo))]]
        data[('Service Producing Supersector Employment', '{}'.format(year), '{}'.format(geo))] = sum(thelist)
        
        #Snapshot, 4 Categories and B&W Collar
        thelist = [data[("Federal Civilian", '{}'.format(year), '{}'.format(geo))], 
                   data[('Federal Military', '{}'.format(year), '{}'.format(geo))], 
                   data[('State & Local Government', '{}'.format(year), '{}'.format(geo))]]
        data[('Snapshot 5: Government Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000
        thelist = [data[("Retail Trade", '{}'.format(year), '{}'.format(geo))], 
                   data[('Accommodation & Food Services', '{}'.format(year), '{}'.format(geo))]]
        data[('Snapshot 5: Retail & Hospitality Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000
        thelist = [data[("Information", '{}'.format(year), '{}'.format(geo))], 
                   data[('Finance & Insurance', '{}'.format(year), '{}'.format(geo))], 
                  data[("Real Estate, Rental, & Leasing", '{}'.format(year), '{}'.format(geo))],
                  data[("Professional & Technical Services", '{}'.format(year), '{}'.format(geo))],
                  data[("Management of Companies & Enterprises", '{}'.format(year), '{}'.format(geo))],
                  data[("Administrative & Waste Services", '{}'.format(year), '{}'.format(geo))],
                  data[("Educational Services", '{}'.format(year), '{}'.format(geo))],
                  data[("Healthcare & Social Assistance", '{}'.format(year), '{}'.format(geo))],
                  data[("Arts, Entertainment, & Recreation", '{}'.format(year), '{}'.format(geo))],
                  data[("Other", '{}'.format(year), '{}'.format(geo))]]
        data[('Snapshot 5: Professional Services Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000
        thelist = [
                   data[('Forestry, Fishing, & Related', '{}'.format(year), '{}'.format(geo))], 
                  data[("Mining", '{}'.format(year), '{}'.format(geo))],
                  data[("Utilities", '{}'.format(year), '{}'.format(geo))],
                  data[("Construction", '{}'.format(year), '{}'.format(geo))],
                  data[("Manufacturing", '{}'.format(year), '{}'.format(geo))],
                  data[("Wholesale Trade", '{}'.format(year), '{}'.format(geo))],
                  data[("Transportation & Warehousing", '{}'.format(year), '{}'.format(geo))]]
        data[('Snapshot 5: Industrial, Manufacturing, Utilities, & Logistics Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000
        data[('Snapshot 5: Farm Employment', '{}'.format(year), '{}'.format(geo))] = data[("Farm", '{}'.format(year), '{}'.format(geo))] * 1000
        #blue collar
        thelist = [data[("Mining", '{}'.format(year), '{}'.format(geo))],
                  data[("Utilities", '{}'.format(year), '{}'.format(geo))],
                  data[("Construction", '{}'.format(year), '{}'.format(geo))],
                  data[("Manufacturing", '{}'.format(year), '{}'.format(geo))],
                  data[("Wholesale Trade", '{}'.format(year), '{}'.format(geo))],
                  data[("Transportation & Warehousing", '{}'.format(year), '{}'.format(geo))]]
        data[('Snapshot: Blue Collar Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000
        thelist = [data[("Information", '{}'.format(year), '{}'.format(geo))],
                  data[("Finance & Insurance", '{}'.format(year), '{}'.format(geo))],
                  data[("Real Estate, Rental, & Leasing", '{}'.format(year), '{}'.format(geo))],
                  data[("Professional & Technical Services", '{}'.format(year), '{}'.format(geo))],
                  data[("Management of Companies & Enterprises", '{}'.format(year), '{}'.format(geo))],
                  data[("Educational Services", '{}'.format(year), '{}'.format(geo))], 
                  data[("Healthcare & Social Assistance", '{}'.format(year), '{}'.format(geo))],
                  data[("Management of Companies & Enterprises", '{}'.format(year), '{}'.format(geo))], 
                  data[("Federal Civilian", '{}'.format(year), '{}'.format(geo))], 
                  data[('Federal Military', '{}'.format(year), '{}'.format(geo))], 
                  data[('State & Local Government', '{}'.format(year), '{}'.format(geo))]]
        data[('Snapshot: White Collar Employment', '{}'.format(year), '{}'.format(geo))] = (sum(thelist)) * 1000

KeyError: ('Total', '1980', 'United States')

In [136]:
data = data.drop(columns = cols)

In [137]:
thing = data.transpose().reset_index(drop = False)

In [138]:
percents = data

In [139]:
for year in years:
    for geo in geos:
        percents[('Farm Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Farm Employment', '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Natural Resources & Mining Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Natural Resources & Mining Employment'
                                                                                                                     , '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Construction Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Construction Employment', '{}'.format(year),
                                                                                                       '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Manufacturing Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Manufacturing Employment', '{}'.format(year),
                                                                                                        '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Trade, Transportation, & Utilities Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Trade, Transportation, & Utilities Employment', '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Information Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Information Employment', '{}'.format(year),
                                                                                                      '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Financial Activities Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Financial Activities Employment',
                                                                                                               '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Professional & Business Services Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Professional & Business Services Employment', 
                                                                                                                       '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Education & Health Services Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Education & Health Services Employment',
                                                                                                                  '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Leisure & Hospitality Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Leisure & Hospitality Employment', '{}'.format(year),
                                                                                                                '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Other Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Other Employment', '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Public Administration Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Public Administration Employment', '{}'.format(year),
                                                                                                                '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Goods Producing Supersector Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Goods Producing Supersector Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Service Producing Supersector Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Service Producing Supersector Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        
        percents[('Snapshot 5: Government Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot 5: Government Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Snapshot 5: Retail & Hospitality Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot 5: Retail & Hospitality Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Snapshot 5: Professional Services Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot 5: Professional Services Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Snapshot 5: Industrial, Manufacturing, Utilities, & Logistics Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot 5: Industrial, Manufacturing, Utilities, & Logistics Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Snapshot 5: Farm Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot 5: Farm Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Snapshot: Blue Collar Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot: Blue Collar Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])
        percents[('Snapshot: White Collar Employment', '{}'.format(year), '{}'.format(geo))] = percent(percents[('Snapshot: White Collar Employment',
                                                                                                                      '{}'.format(year), '{}'.format(geo))], 
                                                                                   percents[('Total Employment', '{}'.format(year), '{}'.format(geo))])

In [140]:
percents = percents.transpose().reset_index(drop = False)

In [141]:
percents['% of Employment'] = percents['Employment']
percents = percents.drop(columns = 'Employment')
percents.head()

,Industry,Year,NAME,% of Employment
0,Total Employment,1980,United States,1.141185e+08
1,Farm Employment,1980,United States,3.328121e+00
2,Natural Resources & Mining Employment,1980,United States,4.666093e+00
3,Construction Employment,1980,United States,5.136616e+00
4,Manufacturing Employment,1980,United States,1.707411e+01


In [142]:
percents = percents.set_index('Industry').transpose()
percents = percents.drop(columns = 'Total Employment')

In [143]:
percents = percents.transpose().reset_index(drop = False)

In [144]:
percents.head()

,Industry,Year,NAME,% of Employment
0,Farm Employment,1980,United States,3.328121
1,Natural Resources & Mining Employment,1980,United States,4.666093
2,Construction Employment,1980,United States,5.136616
3,Manufacturing Employment,1980,United States,17.074114
4,"Trade, Transportation, & Utilities Employment",1980,United States,19.087071


In [145]:
thing.head()

,Industry,Year,NAME,Employment
0,Total Employment,1980,United States,114118464.0
1,Farm Employment,1980,United States,3798000.0
2,Natural Resources & Mining Employment,1980,United States,5324874.0
3,Construction Employment,1980,United States,5861827.0
4,Manufacturing Employment,1980,United States,19484717.0


In [146]:
data = thing.merge(percents, on = ['Industry', 'Year', 'NAME'], how = 'outer')
data.head()

,Industry,Year,NAME,Employment,% of Employment
0,Total Employment,1980,United States,114118464.0,NaN
1,Farm Employment,1980,United States,3798000.0,3.328121
2,Natural Resources & Mining Employment,1980,United States,5324874.0,4.666093
3,Construction Employment,1980,United States,5861827.0,5.136616
4,Manufacturing Employment,1980,United States,19484717.0,17.074114


In [147]:
data.tail(6)

,Industry,Year,NAME,Employment,% of Employment
19640,Snapshot 5: Retail & Hospitality Employment,2045,GNRC Region,402816.0,19.025066
19641,Snapshot 5: Professional Services Employment,2045,GNRC Region,1202164.0,56.778402
19642,"Snapshot 5: Industrial, Manufacturing, Utiliti...",2045,GNRC Region,326601.0,15.425419
19643,Snapshot 5: Farm Employment,2045,GNRC Region,13814.0,0.652437
19644,Snapshot: Blue Collar Employment,2045,GNRC Region,323279.0,15.26852
19645,Snapshot: White Collar Employment,2045,GNRC Region,1059248.0,50.028456


In [148]:
data = data.set_index('Industry').transpose()
data = data.rename(columns = {'Total Employment': 'Ind:Total', 
                              'Farm Employment': 'Ind:Farm',
                              'Natural Resources & Mining Employment': 'Ind:Natural Resources & Mining', 
                              'Construction Employment': 'Ind:Construction', 
                              'Manufacturing Employment': 'Ind:Manufacturing', 
                              'Trade, Transportation, & Utilities Employment': 'Ind:Trade, Transportation, & Utilities', 
                              'Information Employment': 'Ind:Information', 
                              'Financial Activities Employment': 'Ind:Financial Activities', 
                              'Professional & Business Services Employment': 'Ind:Professional & Business Services', 
                              'Education & Health Services Employment': 'Ind:Education & Health Services', 
                              'Leisure & Hospitality Employment': 'Ind:Leisure & Hospitality', 
                              'Public Administration Employment': 'Ind:Public Administration',
                              'Other Employment': 'Ind:Other', 
                              'Goods Producing Supersector Employment': 'Ind:Goods Producing', 
                              'Service Producing Supersector Employment': 'Ind:Service Producing', 
                              'Snapshot 5: Government Employment': 'Ind:Snapshot: Government', 
                              'Snapshot 5: Retail & Hospitality Employment': 'Ind:Snapshot: Retail & Hospitality', 
                              'Snapshot 5: Professional Services Employment': 'Ind:Snapshot: Professional Services', 
                              'Snapshot 5: Industrial, Manufacturing, Utilities, & Logistics Employment': 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
                              'Snapshot 5: Farm Employment': 'Ind:Snapshot: Farm',
                              'Snapshot: Blue Collar Employment': 'Ind:Snapshot: Blue Collar', 
                              'Snapshot: White Collar Employment': 'Ind:Snapshot: White Collar'
                             }).transpose().reset_index(drop = False)


In [149]:
data.tail(50)

,Industry,Year,NAME,Employment,% of Employment
19596,Ind:Snapshot: Retail & Hospitality,2045,MPO,353613.0,18.476903
19597,Ind:Snapshot: Professional Services,2045,MPO,1111410.0,58.073133
19598,"Ind:Snapshot: Industrial, Manufacturing, Utili...",2045,MPO,292883.0,15.303653
19599,Ind:Snapshot: Farm,2045,MPO,9546.0,0.498795
19600,Ind:Snapshot: Blue Collar,2045,MPO,290303.0,15.168844
19601,Ind:Snapshot: White Collar,2045,MPO,977609.0,51.081794
19602,Ind:Total,2045,GNRC,2049144.0,NaN
19603,Ind:Farm,2045,GNRC,12201.0,0.595419
19604,Ind:Natural Resources & Mining,2045,GNRC,18742.0,0.914626
19605,Ind:Construction,2045,GNRC,103264.0,5.039373


In [150]:
data['NAME'].unique()

array(['United States', 'Tennessee', 'Cheatham County, Tennessee',
       'Davidson County, Tennessee', 'Dickson County, Tennessee',
       'Houston County, Tennessee', 'Humphreys County, Tennessee',
       'Maury County, Tennessee', 'Montgomery County, Tennessee',
       'Robertson County, Tennessee', 'Rutherford County, Tennessee',
       'Stewart County, Tennessee', 'Sumner County, Tennessee',
       'Trousdale County, Tennessee', 'Williamson County, Tennessee',
       'Wilson County, Tennessee', 'MPO', 'GNRC', 'GNRC Region'],
      dtype=object)

In [151]:
data['Year'].unique()

array(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2025', '2030', '2035', '2040', '2045'],
      dtype=object)

## Split off projections of total employment for Unites States and Tennessee to append to projections dataframe because they're not included in the next import, and split apart historical - 2020 and 2017 - projection data for county and above geo level for industry employment

In [152]:
#list projection years
years = ['2017', '2020', '2022', '2025', '2035', '2045']

In [153]:
stateusmetrics = data.loc[(data['NAME'] == 'United States')|(data['NAME'] == 'Tennessee')|(data['NAME'] == 'GNRC')|(data['NAME'] == 'GNRC Region')|(data['NAME'] == 'MPO')]
stateusmetrics = stateusmetrics.loc[(data['Industry'] == 'Ind:Total')&(data['Year'].isin(years))].reset_index(drop = True)

In [154]:
stateusmetrics = stateusmetrics.drop(columns = 'Industry').rename(columns = {'Employment': 'Employment Projected'})

In [155]:
stateusmetrics = stateusmetrics.pivot_table(values = 'Employment Projected', index = 'NAME', columns = 'Year')

In [156]:
stateusmetrics.head()

Year,2017,2020,2025,2035,2045
NAME,,,,,
GNRC,1283852.0,1358477.0,1491435.0,1768120.0,2049144.0
GNRC Region,1331775.0,1408578.0,1545335.0,1829481.0,2117291.0
MPO,1198339.0,1267947.0,1392216.0,1651483.0,1913811.0
Tennessee,3977194.0,4150233.0,4451424.0,5040511.0,5592561.0
United States,195849192.0,204186615.0,218269970.0,245561730.0,271456367.0


In [157]:
stateusmetricswide = stateusmetrics

In [158]:
stateusmetrics = stateusmetrics.reset_index(drop = False)

In [159]:
stateusmetrics = stateusmetrics.rename_axis(None, axis=1)  

In [160]:
#make sure this looks correct to add onto the counties and places
stateusmetrics.head()

,NAME,2017,2020,2025,2035,2045
0,GNRC,1283852.0,1358477.0,1491435.0,1768120.0,2049144.0
1,GNRC Region,1331775.0,1408578.0,1545335.0,1829481.0,2117291.0
2,MPO,1198339.0,1267947.0,1392216.0,1651483.0,1913811.0
3,Tennessee,3977194.0,4150233.0,4451424.0,5040511.0,5592561.0
4,United States,195849192.0,204186615.0,218269970.0,245561730.0,271456367.0


### That looks correct to add onto the WP/URBANSIM total employment projections, but now split off historical and disaggregated insdustry projections and calculate change metrics.

In [161]:
#list projection years
years = ['2017', '2020', '2022', '2025', '2035', '2045']
disaggproj = data.loc[data['Year'].isin(years)].reset_index(drop = True)

In [162]:
disaggproj.head()

,Industry,Year,NAME,Employment,% of Employment
0,Ind:Total,2017,United States,195849192.0,NaN
1,Ind:Farm,2017,United States,2665651.0,1.361073
2,Ind:Natural Resources & Mining,2017,United States,5179823.0,2.644802
3,Ind:Construction,2017,United States,10495756.0,5.359101
4,Ind:Manufacturing,2017,United States,13286319.0,6.783954


In [163]:
disaggproj.tail(7)

,Industry,Year,NAME,Employment,% of Employment
2083,Ind:Snapshot: Government,2045,GNRC Region,171896.0,8.118676
2084,Ind:Snapshot: Retail & Hospitality,2045,GNRC Region,402816.0,19.025066
2085,Ind:Snapshot: Professional Services,2045,GNRC Region,1202164.0,56.778402
2086,"Ind:Snapshot: Industrial, Manufacturing, Utili...",2045,GNRC Region,326601.0,15.425419
2087,Ind:Snapshot: Farm,2045,GNRC Region,13814.0,0.652437
2088,Ind:Snapshot: Blue Collar,2045,GNRC Region,323279.0,15.26852
2089,Ind:Snapshot: White Collar,2045,GNRC Region,1059248.0,50.028456


In [164]:
#go long to wide
disaggproj = pd.pivot(disaggproj, index = ['NAME', 'Year'], columns = 'Industry', values = ['Employment', '% of Employment'])
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
disaggproj.columns = disaggproj.columns.to_flat_index()

In [165]:
#rename
inddict = {('Employment', 'Ind:Education & Health Services'): 'Ind:Education & Health Services', 
           ('Employment', 'Ind:Farm'): 'Ind:Farm', 
           ('Employment', 'Ind:Construction'): 'Ind:Construction', 
           ('Employment', 'Ind:Financial Activities'): 'Ind:Financial Activities',
           ('Employment', 'Ind:Information'): 'Ind:Information', 
           ('Employment', 'Ind:Manufacturing'): 'Ind:Manufacturing', 
           ('Employment', 'Ind:Other/Unclassified'): 'Ind:Other/Unclassified', 
           ('Employment', 'Ind:Public Administration'): 'Ind:Public Administration', 
           ('Employment', 'Ind:Total'): 'Ind:Total', 
           ('Employment', 'Ind:Other'): 'Ind:Other', 
           ('Employment', 'Ind:Goods Producing'): 'Ind:Goods Producing', 
           ('Employment', 'Ind:Leisure & Hospitality'): 'Ind:Leisure & Hospitality', 
           ('Employment', 'Ind:Natural Resources & Mining'): 'Ind:Natural Resources & Mining', 
           ('Employment', 'Ind:Professional & Business Services'): 'Ind:Professional & Business Services', 
           ('Employment', 'Ind:Service Producing'): 'Ind:Service Producing', 
           ('Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind:Trade, Transportation, & Utilities',
           
           ('Employment', 'Ind:Snapshot: Government'): 'Ind:Snapshot: Government', 
           ('Employment', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
           ('Employment', 'Ind:Snapshot: Professional Services'): 'Ind:Snapshot: Professional Services', 
           ('Employment', 'Ind:Snapshot: Retail & Hospitality'): 'Ind:Snapshot: Retail & Hospitality', 
           ('Employment', 'Ind:Snapshot: Farm'): 'Ind:Snapshot: Farm', 
           ('Employment', 'Ind:Snapshot: Blue Collar'): 'Ind:Snapshot: Blue Collar', 
           ('Employment', 'Ind:Snapshot: White Collar'): 'Ind:Snapshot: White Collar',
           
           ('% of Employment', 'Ind:Education & Health Services'): 'Ind%:Education & Health Services',
           ('% of Employment', 'Ind:Construction'): 'Ind%:Construction', 
           ('% of Employment', 'Ind:Financial Activities'): 'Ind%:Financial Activities',
           ('% of Employment', 'Ind:Information'): 'Ind%:Information', 
           ('% of Employment', 'Ind:Manufacturing'): 'Ind%:Manufacturing', 
           ('% of Employment', 'Ind:Other/Unclassified'): 'Ind%:Other/Unclassified', 
           ('% of Employment', 'Ind:Public Administration'): 'Ind%:Public Administration', 
           ('% of Employment', 'Ind:Total'): 'Ind%:Total', 
           ('% of Employment', 'Ind:Farm'): 'Ind%:Farm', 
           ('% of Employment', 'Ind:Other'): 'Ind%:Other', 
           ('% of Employment', 'Ind:Goods Producing'): 'Ind%:Goods Producing', 
           ('% of Employment', 'Ind:Leisure & Hospitality'): 'Ind%:Leisure & Hospitality', 
           ('% of Employment', 'Ind:Natural Resources & Mining'): 'Ind%:Natural Resources & Mining', 
           ('% of Employment', 'Ind:Professional & Business Services'): 'Ind%:Professional & Business Services', 
           ('% of Employment', 'Ind:Service Producing'): 'Ind%:Service Producing', 
           ('% of Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind%:Trade, Transportation, & Utilities', 
           
           ('% of Employment', 'Ind:Snapshot: Government'): 'Ind%:Snapshot: Government', 
           ('% of Employment', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
           ('% of Employment', 'Ind:Snapshot: Professional Services'): 'Ind%:Snapshot: Professional Services', 
           ('% of Employment', 'Ind:Snapshot: Retail & Hospitality'): 'Ind%:Snapshot: Retail & Hospitality', 
           ('% of Employment', 'Ind:Snapshot: Farm'): 'Ind%:Snapshot: Farm', 
           ('% of Employment', 'Ind:Snapshot: Blue Collar'): 'Ind%:Snapshot: Blue Collar', 
           ('% of Employment', 'Ind:Snapshot: White Collar'): 'Ind%:Snapshot: White Collar'
           }
disaggproj = disaggproj.rename(columns = inddict)
disaggproj = disaggproj.reset_index(drop = False)

In [166]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
disaggproj['Year'] = disaggproj['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(disaggproj['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [167]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(disaggproj.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = disaggproj.pivot_table(index = ['NAME'], columns = ['Year'], values = cols)
df_pivot.head(2)

Ind%:Construction                                             Ind%:Education & Health Services                        \
Year                                    2017       2020       2025       2035       2045                             2017       2020       2025   
NAME                                                                                                                                              
Cheatham County, Tennessee         11.525833  11.954703  12.463855  12.234989  11.751379                         5.709308   5.842234   6.084337   
Davidson County, Tennessee          5.171798   5.332155   5.493299   5.244603   4.961824                        18.001224  18.747254  20.045723   

                                                 Ind%:Farm                                         Ind%:Financial Activities                      \
Year                             2035       2045      2017      2020      2025      2035      2045                      2017      2020      2025   
NAME                                                                                                                                               
Cheatham County, Tennessee   6.646167   7.180851  2.854654  2.843907  2.801205  2.664940  2.462569                  5.689205  5.726419  5.783133   
Davidson County, Tennessee  22.607361  24.351496  0.071559  0.068851  0.064537  0.056534  0.049961                  9.440628  9.539178  9.618496   

                                               Ind%:Goods Producing                                            Ind%:Information                      \
Year                            2035      2045                 2017      2020       2025       2035       2045             2017      2020      2025   
NAME                                                                                                                                                  
Cheatham County, Tennessee  5.998813  6.279551            31.025933  30.96770  30.536145  28.580676  26.556344         0.797427  0.759233  0.704819   
Davidson County, Tennessee  9.624041  9.615558             8.898039   8.86707   8.700569   7.886342   7.185069         2.593410  2.506638  2.364828   

                                               Ind%:Leisure & Hospitality                                             Ind%:Manufacturing             \
Year                            2035      2045                       2017       2020       2025       2035       2045               2017       2020   
NAME                                                                                                                                                  
Cheatham County, Tennessee  0.609592  0.541765                   8.684581   8.846995   9.301205  10.503318  11.578999          15.881525  15.409857   
Davidson County, Tennessee  2.128992  1.966735                  12.473892  12.364387  12.336694  12.463459  12.576184           3.441620   3.257345   

                                                            Ind%:Natural Resources & Mining                                        Ind%:Other  \
Year                             2025       2035       2045                            2017     2020      2025      2035      2045       2017   
NAME                                                                                                                                            
Cheatham County, Tennessee  14.518072  12.925500  11.578999                        3.618575  3.60314  3.554217  3.420187  3.225965   7.016016   
Davidson County, Tennessee   2.940910   2.393496   1.985470                        0.284621  0.27757  0.266360  0.248243  0.237775   5.696296   

                                                                   Ind%:Professional & Business Services                                   \
Year                            2020      2025      2035      2045                                  2017       2020       2025       2035   
NAME                          

In [168]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Ind%:Construction                                             Ind%:Education & Health Services                        \
                                        2017       2020       2025       2035       2045                             2017       2020       2025   
                                        None       None       None       None       None                             None       None       None   
NAME                                                                                                                                              
Cheatham County, Tennessee         11.525833  11.954703  12.463855  12.234989  11.751379                         5.709308   5.842234   6.084337   
Davidson County, Tennessee          5.171798   5.332155   5.493299   5.244603   4.961824                        18.001224  18.747254  20.045723   
Dickson County, Tennessee           8.661355   9.423150  10.405281  10.071618   9.203792                        10.932731  10.876660  10.838243   

                                                 Ind%:Farm                                         Ind%:Financial Activities                      \
                                 2035       2045      2017      2020      2025      2035      2045                      2017      2020      2025   
                                 None       None      None      None      None      None      None                      None      None      None   
NAME                                                                                                                                               
Cheatham County, Tennessee   6.646167   7.180851  2.854654  2.843907  2.801205  2.664940  2.462569                  5.689205  5.726419  5.783133   
Davidson County, Tennessee  22.607361  24.351496  0.071559  0.068851  0.064537  0.056534  0.049961                  9.440628  9.539178  9.618496   
Dickson County, Tennessee   11.065230  11.344983  4.447616  4.356736  4.212506  4.019614  3.870499                  6.445475  6.466793  6.529917   

                                               Ind%:Goods Producing                                             Ind%:Information            \
                                2035      2045                 2017       2020       2025       2035       2045             2017      2020   
                                None      None                 None       None       None       None       None             None      None   
NAME                                                                                                                                         
Cheatham County, Tennessee  5.998813  6.279551            31.025933  30.967700  30.536145  28.580676  26.556344         0.797427  0.759233   
Davidson County, Tennessee  9.624041  9.615558             8.898039   8.867070   8.700569   7.886342   7.185069         2.593410  2.506638   
Dickson County, Tennessee   6.852055  7.289742            28.727157  28.865275  28.788416  26.872701  24.846301         0.931542  0.891841   

                                                         Ind%:Leisure & Hospitality                                             Ind%:Manufacturing  \
                                2025      2035      2045                       2017       2020       2025       2035       2045               2017   
                                None      None      None                       None       None       None       None       None               None   
NAME                                                                                                                                                 
Cheatham County, Tennessee  0.704819  0.609592  0.541765                   8.684581   8.846995   9.301205  10.503318  11.578999          15.881525   
Davidson County, Tennessee  2.364828  2.128992  1.966735                  12.473892  12.364387  12.336694  12.463459  12.576184           3.441620   
Dickson County, Tennessee   0.841082  0.771017  0.726854         

In [169]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
# first_level.remove('NAME')
# first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [170]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
disaggproj = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [171]:
#reformat and rename columns
disaggproj = disaggproj.stack([1, 1])
disaggproj = disaggproj.reset_index(drop = False)
disaggproj = disaggproj.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [172]:
#map to geoid dictionary from module and add source
disaggproj['GEO_ID'] = disaggproj['NAME'].map(geotogeoid)
disaggproj['Source'] = 'Woods & Poole 2017 Base'

In [173]:
#final check
disaggproj.head()

,NAME,Year,Time Frame,Ind%:Construction,Ind%:Education & Health Services,Ind%:Farm,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: Farm,Ind%:Snapshot: Government,"Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot: Professional Services,Ind%:Snapshot: Retail & Hospitality,Ind%:Snapshot: White Collar,"Ind%:Trade, Transportation, & Utilities",Ind:Construction,Ind:Construction % Change,Ind:Construction Change,Ind:Education & Health Services,Ind:Education & Health Services % Change,Ind:Education & Health Services Change,Ind:Farm,Ind:Farm % Change,Ind:Farm Change,Ind:Financial Activities,Ind:Financial Activities % Change,Ind:Financial Activities Change,Ind:Goods Producing,Ind:Goods Producing % Change,Ind:Goods Producing Change,Ind:Information,Ind:Information % Change,Ind:Information Change,Ind:Leisure & Hospitality,Ind:Leisure & Hospitality % Change,Ind:Leisure & Hospitality Change,Ind:Manufacturing,Ind:Manufacturing % Change,Ind:Manufacturing Change,Ind:Natural Resources & Mining,Ind:Natural Resources & Mining % Change,Ind:Natural Resources & Mining Change,Ind:Other,Ind:Other % Change,Ind:Other Change,Ind:Professional & Business Services,Ind:Professional & Business Services % Change,Ind:Professional & Business Services Change,Ind:Public Administration,Ind:Public Administration % Change,Ind:Public Administration Change,Ind:Service Producing,Ind:Service Producing % Change,Ind:Service Producing Change,Ind:Snapshot: Blue Collar,Ind:Snapshot: Blue Collar % Change,Ind:Snapshot: Blue Collar Change,Ind:Snapshot: Farm,Ind:Snapshot: Farm % Change,Ind:Snapshot: Farm Change,Ind:Snapshot: Government,Ind:Snapshot: Government % Change,Ind:Snapshot: Government Change,"Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics % Change","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics Change",Ind:Snapshot: Professional Services,Ind:Snapshot: Professional Services % Change,Ind:Snapshot: Professional Services Change,Ind:Snapshot: Retail & Hospitality,Ind:Snapshot: Retail & Hospitality % Change,Ind:Snapshot: Retail & Hospitality Change,Ind:Snapshot: White Collar,Ind:Snapshot: White Collar % Change,Ind:Snapshot: White Collar Change,Ind:Total,Ind:Total % Change,Ind:Total Change,"Ind:Trade, Transportation, & Utilities","Ind:Trade, Transportation, & Utilities % Change","Ind:Trade, Transportation, & Utilities Change",GEO_ID,Source
0,"Cheatham County, Tennessee",2017,None,11.525833,5.709308,2.854654,5.689205,31.025933,0.797427,8.684581,15.881525,3.618575,7.016016,12.242847,12.001608,52.140990,34.222341,2.854654,12.001608,34.818736,35.401729,14.923273,30.362528,16.833076,1720.0,NaN,NaN,852.0,NaN,NaN,426.0,NaN,NaN,849.0,NaN,NaN,4630.0,NaN,NaN,119.0,NaN,NaN,1296.0,NaN,NaN,2370.0,NaN,NaN,540.0,NaN,NaN,1047.0,NaN,NaN,1827.0,NaN,NaN,1791.0,NaN,NaN,7781.0,NaN,NaN,5107.0,NaN,NaN,426.0,NaN,NaN,1791.0,NaN,NaN,5196.0,NaN,NaN,5283.0,NaN,NaN,2227.0,NaN,NaN,4531.0,NaN,NaN,14923.0,NaN,NaN,2512.0,NaN,NaN,0500000US47021,Woods & Poole 2017 Base
1,"Cheatham County, Tennessee",2020,None,11.954703,5.842234,2.843907,5.726419,30.967700,0.759233,8.846995,15.409857,3.603140,7.109767,12.308583,11.729507,52.322738,34.004633,2.843907,11.729507,34.596577,35.780466,15.049543,30.195599,16.709561,1858.0,NaN,NaN,908.0,NaN,NaN,442.0,NaN,NaN,890.0,NaN,NaN,4813.0,NaN,NaN,118.0,NaN,NaN,1375.0,NaN,NaN,2395.0,NaN,NaN,560.0,NaN,NaN,1105.0,NaN,NaN,1913.0,NaN,NaN,1823.0,NaN,NaN,8132.0,NaN,NaN,5285.0,NaN,NaN,442.0,NaN,NaN,1823.0,NaN,NaN,5377.0,NaN,NaN,5561.0,NaN,NaN,2339.0,NaN,NaN,4693.0,NaN,NaN,15542.0,NaN,NaN,2597.0,NaN,NaN,0500000US47021,Woods & Poole 2017 Base
2,"Cheatham County, Tennessee",2025,None,12.463855,6.084337,2.801205,5.783133,30.536145,0.7

In [174]:
disaggproj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 92 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   NAME                                                                      285 non-null    object 
 1   Year                                                                      285 non-null    object 
 2   Time Frame                                                                285 non-null    object 
 3   Ind%:Construction                                                         95 non-null     float64
 4   Ind%:Education & Health Services                                          95 non-null     float64
 5   Ind%:Farm                                                                 95 non-null     float64
 6   Ind%:Financial Activities                                         

In [175]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
disaggproj.to_sql('WP_IndustryEmployment_CurrentandProjected_Annual_Change_2017Base', conn, if_exists = 'replace', index = False)

285

### Now do the same with the historical data

In [176]:
histyears = ['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
             '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
             '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
             '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
             '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

In [177]:
disagghist = data.loc[data['Year'].isin(histyears)]

In [178]:
#go long to wide
disagghist = pd.pivot(disagghist, index = ['NAME', 'Year'], columns = 'Industry', values = ['Employment', '% of Employment'])
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
disagghist.columns = disagghist.columns.to_flat_index()

In [179]:
#rename
inddict = {('Employment', 'Ind:Education & Health Services'): 'Ind:Education & Health Services', 
           ('Employment', 'Ind:Farm'): 'Ind:Farm', 
           ('Employment', 'Ind:Construction'): 'Ind:Construction', 
           ('Employment', 'Ind:Financial Activities'): 'Ind:Financial Activities',
           ('Employment', 'Ind:Information'): 'Ind:Information', 
           ('Employment', 'Ind:Manufacturing'): 'Ind:Manufacturing', 
           ('Employment', 'Ind:Other/Unclassified'): 'Ind:Other/Unclassified', 
           ('Employment', 'Ind:Public Administration'): 'Ind:Public Administration', 
           ('Employment', 'Ind:Total'): 'Ind:Total', 
           ('Employment', 'Ind:Other'): 'Ind:Other', 
           ('Employment', 'Ind:Goods Producing'): 'Ind:Goods Producing', 
           ('Employment', 'Ind:Leisure & Hospitality'): 'Ind:Leisure & Hospitality', 
           ('Employment', 'Ind:Natural Resources & Mining'): 'Ind:Natural Resources & Mining', 
           ('Employment', 'Ind:Professional & Business Services'): 'Ind:Professional & Business Services', 
           ('Employment', 'Ind:Service Producing'): 'Ind:Service Producing', 
           ('Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind:Trade, Transportation, & Utilities',
           
           ('Employment', 'Ind:Snapshot: Government'): 'Ind:Snapshot: Government', 
           ('Employment', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
           ('Employment', 'Ind:Snapshot: Professional Services'): 'Ind:Snapshot: Professional Services', 
           ('Employment', 'Ind:Snapshot: Retail & Hospitality'): 'Ind:Snapshot: Retail & Hospitality', 
           ('Employment', 'Ind:Snapshot: Farm'): 'Ind:Snapshot: Farm',
           ('Employment', 'Ind:Snapshot: Blue Collar'): 'Ind:Snapshot: Blue Collar', 
           ('Employment', 'Ind:Snapshot: White Collar'): 'Ind:Snapshot: White Collar',

           
           ('% of Employment', 'Ind:Education & Health Services'): 'Ind%:Education & Health Services',
           ('% of Employment', 'Ind:Construction'): 'Ind%:Construction', 
           ('% of Employment', 'Ind:Financial Activities'): 'Ind%:Financial Activities',
           ('% of Employment', 'Ind:Information'): 'Ind%:Information', 
           ('% of Employment', 'Ind:Manufacturing'): 'Ind%:Manufacturing', 
           ('% of Employment', 'Ind:Other/Unclassified'): 'Ind%:Other/Unclassified', 
           ('% of Employment', 'Ind:Public Administration'): 'Ind%:Public Administration', 
           ('% of Employment', 'Ind:Total'): 'Ind%:Total', 
           ('% of Employment', 'Ind:Farm'): 'Ind%:Farm', 
           ('% of Employment', 'Ind:Other'): 'Ind%:Other', 
           ('% of Employment', 'Ind:Goods Producing'): 'Ind%:Goods Producing', 
           ('% of Employment', 'Ind:Leisure & Hospitality'): 'Ind%:Leisure & Hospitality', 
           ('% of Employment', 'Ind:Natural Resources & Mining'): 'Ind%:Natural Resources & Mining', 
           ('% of Employment', 'Ind:Professional & Business Services'): 'Ind%:Professional & Business Services', 
           ('% of Employment', 'Ind:Service Producing'): 'Ind%:Service Producing', 
           ('% of Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind%:Trade, Transportation, & Utilities', 
           

           ('% of Employment', 'Ind:Snapshot: Government'): 'Ind%:Snapshot: Government', 
           ('% of Employment', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
           ('% of Employment', 'Ind:Snapshot: Professional Services'): 'Ind%:Snapshot: Professional Services', 
           ('% of Employment', 'Ind:Snapshot: Retail & Hospitality'): 'Ind%:Snapshot: Retail & Hospitality', 
           ('% of Employment', 'Ind:Snapshot: Farm'): 'Ind%:Snapshot: Farm', 
           ('% of Employment', 'Ind:Snapshot: Blue Collar'): 'Ind%:Snapshot: Blue Collar', 
           ('% of Employment', 'Ind:Snapshot: White Collar'): 'Ind%:Snapshot: White Collar'
           
           }
disagghist = disagghist.rename(columns = inddict)
disagghist = disagghist.reset_index(drop = False)

In [180]:
#make sure year is formatted as an integer and generate list of years and all possible time frames
disagghist['Year'] = disagghist['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(disagghist['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [181]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(disagghist.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = disagghist.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Ind%:Construction                                                                                                 \
Year                                    1980      1981      1982      1983      1984       1985       1986       1987       1988       1989   
NAME                                                                                                                                          
Cheatham County, Tennessee          6.685850  6.743208  6.661031  7.764901  9.607057  10.256832  11.472463  13.188518  11.875824  11.471497   
Davidson County, Tennessee          5.564207  5.179203  5.037608  5.192911  5.659482   5.859804   6.100985   5.997186   5.531208   5.341912   

                                                                                                                                                     \
Year                             1990       1991       1992       1993       1994       1995       1996       1997       1998       1999       2000   
NAME                                                                                                                                                  
Cheatham County, Tennessee  11.150169  12.012729  10.946102  11.067335  10.957110  11.565745  11.692584  12.146295  13.953899  14.609487  13.909031   
Davidson County, Tennessee   4.978360   4.514223   4.422530   4.698062   4.978074   5.045086   5.142868   5.280652   5.181487   5.052889   5.037205   

                                                                                                                                                     \
Year                             2001       2002       2003       2004       2005       2006       2007       2008       2009       2010       2011   
NAME                                                                                                                                                  
Cheatham County, Tennessee  13.099415  11.631514  11.382419  12.196499  12.749983  12.317146  13.372244  11.784467  11.154774  11.073059  11.343652   
Davidson County, Tennessee   5.004035   4.781140   5.031878   4.873361   5.075820   5.301846   5.569788   5.389276   4.765359   4.661956   4.643928   

                                                                                                                               \
Year                             2012       2013       2014       2015       2016       2017       2018       2019       2020   
NAME                                                                                                                            
Cheatham County, Tennessee  10.719281  10.823081  11.477160  11.023785  11.300625  11.525833  11.684621  11.823399  11.954703   
Davidson County, Tennessee   4.656579   4.531028   4.626015   4.961976   5.080895   5.171798   5.232729   5.285479   5.332155   

                           Ind%:Education & Health Services                                                                                    \
Year                                                   1980      1981      1982      1983      1984      1985      1986       1987       1988   
NAME                                                                                                                                            
Cheatham County, Tennessee                         1.760325  1.714101  1.808960  1.854305  2.133119  2.502470  2.859944   3.118697   3.045834   
Davidson County, Tennessee                         8.707408  8.895539  9.233806  9.454425  9.468357  9.650796  9.914036  10.177997  10.553362   

                                                                                                                                                     \
Year                             1989       1990       1991       1992       1993       1994       1995       1996       1997       1998       1999   
NAME                                                                                                                         

In [182]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Ind%:Construction                                                                                                 \
                                        1980      1981      1982      1983      1984       1985       1986       1987       1988       1989   
                                        None      None      None      None      None       None       None       None       None       None   
NAME                                                                                                                                          
Cheatham County, Tennessee          6.685850  6.743208  6.661031  7.764901  9.607057  10.256832  11.472463  13.188518  11.875824  11.471497   
Davidson County, Tennessee          5.564207  5.179203  5.037608  5.192911  5.659482   5.859804   6.100985   5.997186   5.531208   5.341912   
Dickson County, Tennessee           5.346222  5.043038  5.338581  5.067300  6.227351   6.691249   7.297486   7.937551   8.015892   8.215086   

                                                                                                                                                     \
                                 1990       1991       1992       1993       1994       1995       1996       1997       1998       1999       2000   
                                 None       None       None       None       None       None       None       None       None       None       None   
NAME                                                                                                                                                  
Cheatham County, Tennessee  11.150169  12.012729  10.946102  11.067335  10.957110  11.565745  11.692584  12.146295  13.953899  14.609487  13.909031   
Davidson County, Tennessee   4.978360   4.514223   4.422530   4.698062   4.978074   5.045086   5.142868   5.280652   5.181487   5.052889   5.037205   
Dickson County, Tennessee    8.011356   6.657029   7.054790   6.717253   6.739180   7.880613   7.938697   8.330914   8.442888   8.461936   8.684040   

                                                                                                                                                     \
                                 2001       2002       2003       2004       2005       2006       2007       2008       2009       2010       2011   
                                 None       None       None       None       None       None       None       None       None       None       None   
NAME                                                                                                                                                  
Cheatham County, Tennessee  13.099415  11.631514  11.382419  12.196499  12.749983  12.317146  13.372244  11.784467  11.154774  11.073059  11.343652   
Davidson County, Tennessee   5.004035   4.781140   5.031878   4.873361   5.075820   5.301846   5.569788   5.389276   4.765359   4.661956   4.643928   
Dickson County, Tennessee    8.751485   8.493189   9.173935   9.532415  10.177363  10.297304  10.549237  10.266344   9.679985   9.249392   8.147451   

                                                                                                                               \
                                 2012       2013       2014       2015       2016       2017       2018       2019       2020   
                                 None       None       None       None       None       None       None       None       None   
NAME                                                                                                                            
Cheatham County, Tennessee  10.719281  10.823081  11.477160  11.023785  11.300625  11.525833  11.684621  11.823399  11.954703   
Davidson County, Tennessee   4.656579   4.531028   4.626015   4.961976   5.080895   5.171798   5.232729   5.285479   5.332155   
Dickson County, Tennessee    8.233022   7.514747   7.775951   7.853747   8.297066   8.661355   8.935057   9.184695   9.423150   

     

In [183]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
# first_level.remove('NAME')
# first_level.remove('Year')
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [184]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
disagghist = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [185]:
#reformat and rename columns
disagghist = disagghist.stack([1, 1])
disagghist = disagghist.reset_index(drop = False)
disagghist = disagghist.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [186]:
#map to geoid dictionary from module and add source
disagghist['GEO_ID'] = disagghist['NAME'].map(geotogeoid)
disagghist['Source'] = 'Woods & Poole 2017 Base'

In [187]:
#final check
disagghist.head()

,NAME,Year,Time Frame,Ind%:Construction,Ind%:Education & Health Services,Ind%:Farm,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: Farm,Ind%:Snapshot: Government,"Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot: Professional Services,Ind%:Snapshot: Retail & Hospitality,Ind%:Snapshot: White Collar,"Ind%:Trade, Transportation, & Utilities",Ind:Construction,Ind:Construction % Change,Ind:Construction Change,Ind:Education & Health Services,Ind:Education & Health Services % Change,Ind:Education & Health Services Change,Ind:Farm,Ind:Farm % Change,Ind:Farm Change,Ind:Financial Activities,Ind:Financial Activities % Change,Ind:Financial Activities Change,Ind:Goods Producing,Ind:Goods Producing % Change,Ind:Goods Producing Change,Ind:Information,Ind:Information % Change,Ind:Information Change,Ind:Leisure & Hospitality,Ind:Leisure & Hospitality % Change,Ind:Leisure & Hospitality Change,Ind:Manufacturing,Ind:Manufacturing % Change,Ind:Manufacturing Change,Ind:Natural Resources & Mining,Ind:Natural Resources & Mining % Change,Ind:Natural Resources & Mining Change,Ind:Other,Ind:Other % Change,Ind:Other Change,Ind:Professional & Business Services,Ind:Professional & Business Services % Change,Ind:Professional & Business Services Change,Ind:Public Administration,Ind:Public Administration % Change,Ind:Public Administration Change,Ind:Service Producing,Ind:Service Producing % Change,Ind:Service Producing Change,Ind:Snapshot: Blue Collar,Ind:Snapshot: Blue Collar % Change,Ind:Snapshot: Blue Collar Change,Ind:Snapshot: Farm,Ind:Snapshot: Farm % Change,Ind:Snapshot: Farm Change,Ind:Snapshot: Government,Ind:Snapshot: Government % Change,Ind:Snapshot: Government Change,"Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics % Change","Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics Change",Ind:Snapshot: Professional Services,Ind:Snapshot: Professional Services % Change,Ind:Snapshot: Professional Services Change,Ind:Snapshot: Retail & Hospitality,Ind:Snapshot: Retail & Hospitality % Change,Ind:Snapshot: Retail & Hospitality Change,Ind:Snapshot: White Collar,Ind:Snapshot: White Collar % Change,Ind:Snapshot: White Collar Change,Ind:Total,Ind:Total % Change,Ind:Total Change,"Ind:Trade, Transportation, & Utilities","Ind:Trade, Transportation, & Utilities % Change","Ind:Trade, Transportation, & Utilities Change",GEO_ID,Source
0,"Cheatham County, Tennessee",1980,None,6.685850,1.760325,13.761002,5.111713,55.924171,0.846310,4.011510,34.647935,14.590386,3.402167,2.945159,14.201083,32.278267,43.618822,13.761002,14.201083,44.245091,16.350711,11.442112,23.375085,11.797563,395.0,NaN,NaN,104.0,NaN,NaN,813.0,NaN,NaN,302.0,NaN,NaN,3304.0,NaN,NaN,50.0,NaN,NaN,237.0,NaN,NaN,2047.0,NaN,NaN,862.0,NaN,NaN,201.0,NaN,NaN,174.0,NaN,NaN,839.0,NaN,NaN,1907.0,NaN,NaN,2577.0,NaN,NaN,813.0,NaN,NaN,839.0,NaN,NaN,2614.0,NaN,NaN,966.0,NaN,NaN,676.0,NaN,NaN,1381.0,NaN,NaN,5908.0,NaN,NaN,697.0,NaN,NaN,0500000US47021,Woods & Poole 2017 Base
1,"Cheatham County, Tennessee",1981,None,6.743208,1.714101,13.130660,4.802717,56.840233,0.873221,3.783959,35.025873,15.071151,4.349935,2.894567,13.486417,31.904916,44.307891,13.130660,13.486417,46.054334,16.785252,10.543338,22.283312,11.254851,417.0,NaN,NaN,106.0,NaN,NaN,812.0,NaN,NaN,297.0,NaN,NaN,3515.0,NaN,NaN,54.0,NaN,NaN,234.0,NaN,NaN,2166.0,NaN,NaN,932.0,NaN,NaN,269.0,NaN,NaN,179.0,NaN,NaN,834.0,NaN,NaN,1973.0,NaN,NaN,2740.0,NaN,NaN,812.0,NaN,NaN,834.0,NaN,NaN,2848.0,NaN,NaN,1038.0,NaN,NaN,652.0,NaN,NaN,1378.0,NaN,NaN,6184.0,NaN,NaN,696.0,NaN,NaN,0500000US47021,Woods & Poole 2017 Base
2,"Cheatham County, Tennessee",1982,None,6.661031,1.808960,14.150465,4.378698,55.553677,0.845309,3.871513,32.6

In [188]:
disagghist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16359 entries, 0 to 16358
Data columns (total 92 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   NAME                                                                      16359 non-null  object 
 1   Year                                                                      16359 non-null  object 
 2   Time Frame                                                                16359 non-null  object 
 3   Ind%:Construction                                                         779 non-null    float64
 4   Ind%:Education & Health Services                                          779 non-null    float64
 5   Ind%:Farm                                                                 779 non-null    float64
 6   Ind%:Financial Activities                                     

In [189]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
disagghist.to_sql('WP_IndustryEmployment_Historical_Current_Annual_Change_2017Base', conn, if_exists = 'replace', index = False)

16359

### Now bring in the URBANSIM data. This is confirmed to have the 2017 W&P numbers as the model inputs so for simplicity sake we're importing those along with the place level data, and then just joining the US and Tennessee because those were not in the model. The incorporated and unincorporated areas were calculated by hand by research staff as the model allows this to be broken up and more information is available technically... whether we think it's realiable or not is up for question the first time someone wants to use it... but more available than simply aggregating Census areas.

In [190]:
data = pd.read_csv('../../Data Downloads/WoodsPooleUrbanSIM_EmploymentProjections_2017Base.csv')
data = data.append(stateusmetrics)
data = data.drop(columns = 'GEO_ID')
data.head()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_17364\1531925892.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateusmetrics)


,NAME,2017,2020,2025,2035,2045
0,"Cheatham County, Tennessee",14999.0,15598.0,16641.0,18551.0,20304.0
1,"Davidson County, Tennessee",638807.0,663935.0,708262.0,794173.0,864682.0
2,"Dickson County, Tennessee",25580.0,26652.0,28417.0,31114.0,33019.0
3,"Houston County, Tennessee",2606.0,2728.0,2954.0,3365.0,3719.0
4,"Humphreys County, Tennessee",8813.0,9027.0,9419.0,10092.0,10660.0


In [191]:
datawide = data.set_index('NAME').add_prefix('Employment ')

In [192]:
datawide = datawide.reset_index(drop = False)

In [193]:
datawide['GEO_ID'] = datawide['NAME'].map(geotogeoid)
datawide['Source'] = 'Woods & Poole 2017 Base'

In [194]:
#final check
datawide.head()

,NAME,Employment 2017,Employment 2020,Employment 2025,Employment 2035,Employment 2045,GEO_ID,Source
0,"Cheatham County, Tennessee",14999.0,15598.0,16641.0,18551.0,20304.0,0500000US47021,Woods & Poole 2017 Base
1,"Davidson County, Tennessee",638807.0,663935.0,708262.0,794173.0,864682.0,0500000US47037,Woods & Poole 2017 Base
2,"Dickson County, Tennessee",25580.0,26652.0,28417.0,31114.0,33019.0,0500000US47043,Woods & Poole 2017 Base
3,"Houston County, Tennessee",2606.0,2728.0,2954.0,3365.0,3719.0,0500000US47083,Woods & Poole 2017 Base
4,"Humphreys County, Tennessee",8813.0,9027.0,9419.0,10092.0,10660.0,0500000US47085,Woods & Poole 2017 Base


In [195]:
datawide['NAME'].unique()

array(['Cheatham County, Tennessee', 'Davidson County, Tennessee',
       'Dickson County, Tennessee', 'Houston County, Tennessee',
       'Humphreys County, Tennessee', 'Maury County, Tennessee',
       'Montgomery County, Tennessee', 'Robertson County, Tennessee',
       'Rutherford County, Tennessee', 'Stewart County, Tennessee',
       'Sumner County, Tennessee', 'Williamson County, Tennessee',
       'Wilson County, Tennessee', 'Trousdale County, Tennessee',
       'Cheatham Incorporated', 'Davidson Incorporated',
       'Dickson Incorporated', 'Houston Incorporated',
       'Humphreys Incorporated', 'Maury Incorporated',
       'Montgomery Incorporated', 'Robertson Incorporated',
       'Rutherford Incorporated', 'Stewart Incorporated',
       'Sumner Incorporated', 'Williamson Incorporated',
       'Wilson Incorporated', 'Cheatham Unincorporated',
       'Davidson Unincorporated', 'Dickson Unincorporated',
       'Houston Unincorporated', 'Humphreys Unincorporated',
       'Maur

In [196]:
test = datawide.loc[datawide['NAME'] == 'La Vergne city, Tennessee']
test

,NAME,Employment 2017,Employment 2020,Employment 2025,Employment 2035,Employment 2045,GEO_ID,Source
65,"La Vergne city, Tennessee",20548.0,21857.0,24353.0,29319.0,34625.0,1600000US4741200,Woods & Poole 2017 Base


In [197]:
datawide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NAME             101 non-null    object 
 1   Employment 2017  101 non-null    float64
 2   Employment 2020  101 non-null    float64
 3   Employment 2025  101 non-null    float64
 4   Employment 2035  101 non-null    float64
 5   Employment 2045  101 non-null    float64
 6   GEO_ID           68 non-null     object 
 7   Source           101 non-null    object 
dtypes: float64(5), object(3)
memory usage: 6.4+ KB


In [198]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
datawide.to_sql('WPURBANSIM_Employment_CurrentandProjected_Annual_WideFormat_2017Base', conn, if_exists = 'replace', index = False)

101

In [199]:
data = data.melt(id_vars = ['NAME'], var_name = ['Year'], value_vars = ['2017', '2020', '2025', '2035', '2045'], value_name = 'Employment')

In [200]:
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [201]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Employment                                
Year                               2017    2020    2025    2035    2045
NAME                                                                   
Adams city, Tennessee             243.0   268.0   313.0   428.0   589.0
Ashland City town, Tennessee     7858.0  8044.0  8395.0  9019.0  9538.0

In [202]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(2)

Employment                                
                                   2017    2020    2025    2035    2045
                                   None    None    None    None    None
NAME                                                                   
Adams city, Tennessee             243.0   268.0   313.0   428.0   589.0
Ashland City town, Tennessee     7858.0  8044.0  8395.0  9019.0  9538.0

In [203]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [204]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [205]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [206]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Woods & Poole 2017 Base and UrbanSIM'

In [207]:
#final check
data.head()

,NAME,Year,Time Frame,Employment,Employment % Change,Employment Change,GEO_ID,Source
0,"Adams city, Tennessee",2017,None,243.0,NaN,NaN,1600000US4700200,Woods & Poole 2017 Base and UrbanSIM
1,"Adams city, Tennessee",2020,None,268.0,NaN,NaN,1600000US4700200,Woods & Poole 2017 Base and UrbanSIM
2,"Adams city, Tennessee",2025,None,313.0,NaN,NaN,1600000US4700200,Woods & Poole 2017 Base and UrbanSIM
3,"Adams city, Tennessee",2035,None,428.0,NaN,NaN,1600000US4700200,Woods & Poole 2017 Base and UrbanSIM
4,"Adams city, Tennessee",2045,None,589.0,NaN,NaN,1600000US4700200,Woods & Poole 2017 Base and UrbanSIM


In [208]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NAME                 1470 non-null   object 
 1   Year                 1470 non-null   object 
 2   Time Frame           1470 non-null   object 
 3   Employment           490 non-null    float64
 4   Employment % Change  970 non-null    float64
 5   Employment Change    980 non-null    float64
 6   GEO_ID               1020 non-null   object 
 7   Source               1470 non-null   object 
dtypes: float64(3), object(5)
memory usage: 92.0+ KB


In [209]:
data['Year'].unique()

array([2017, 2020, 2025, 2035, 2045, 'None'], dtype=object)

In [210]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('WPURBANSIM_TotalEmployment_CurrentandProjected_Annual_Change_2017Base', conn, if_exists = 'replace', index = False)

1470